In [1]:
!git clone https://github.com/raghav-thiruv/Omdena-Quantum-Self-Driving.git

Cloning into 'Omdena-Quantum-Self-Driving'...
remote: Enumerating objects: 45757, done.
remote: Counting objects: 100% (11667/11667), done.
remote: Compressing objects: 100% (11652/11652), done.
remote: Total 45757 (delta 29), reused 11639 (delta 14), pack-reused 34090
Receiving objects: 100% (45757/45757), 2.16 GiB | 34.07 MiB/s, done.
Resolving deltas: 100% (58/58), done.
Updating files: 100% (45593/45593), done.


In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import pandas as pd
import os

In [3]:
from struct import unpack
from tqdm import tqdm
import os

img_dir = '/content/Omdena-Quantum-Self-Driving/Images/driving_dataset1/'
root_img = '/content/Omdena-Quantum-Self-Driving/Images/driving_dataset1/'
marker_mapping = {
    0xffd8: "Start of Image",
    0xffe0: "Application Default Header",
    0xffdb: "Quantization Table",
    0xffc0: "Start of Frame",
    0xffc4: "Define Huffman Table",
    0xffda: "Start of Scan",
    0xffd9: "End of Image"
}


class JPEG:
    def __init__(self, image_file):
        with open(image_file, 'rb') as f:
            self.img_data = f.read()

    def decode(self):
        data = self.img_data
        while(True):
            marker, = unpack(">H", data[0:2])
            # print(marker_mapping.get(marker))
            if marker == 0xffd8:
                data = data[2:]
            elif marker == 0xffd9:
                return
            elif marker == 0xffda:
                data = data[-2:]
            else:
                lenchunk, = unpack(">H", data[2:4])
                data = data[2+lenchunk:]
            if len(data)==0:
               raise TypeError("issue reading jpeg file")


bads = []

for dirName, subdirList, fileList in os.walk(img_dir):
    imagesList = fileList
    for img in tqdm(imagesList):
      image = os.path.join(root_img,img)
      image = JPEG(image)
      try:
        image.decode()
      except:
        bads.append(img)


for name in bads:
  os.remove(os.path.join(root_img,name))

100%|██████████| 45569/45569 [00:02<00:00, 17003.81it/s]


In [6]:
# Load steering angles from text file
df = pd.read_csv('/content/Omdena-Quantum-Self-Driving/Images/driving_dataset1/data.txt', names=['filename', 'steering_angle'], delimiter=' ')
image_dir = '/content/Omdena-Quantum-Self-Driving/Images/driving_dataset1/'
df['filename'] = df['filename'].apply(lambda x: os.path.join(image_dir, x))
df.head()

,filename,steering_angle
0,/content/Omdena-Quantum-Self-Driving/Images/dr...,0.0
1,/content/Omdena-Quantum-Self-Driving/Images/dr...,0.0
2,/content/Omdena-Quantum-Self-Driving/Images/dr...,0.0
3,/content/Omdena-Quantum-Self-Driving/Images/dr...,0.0
4,/content/Omdena-Quantum-Self-Driving/Images/dr...,0.0


In [7]:
def resnet_model(input_shape):
    base_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    output = tf.keras.layers.Dense(1)(x)
    model = tf.keras.Model(inputs=base_model.input, outputs=output)
    return model

def load_image(image_path):
    try:
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (224, 224))
        image = tf.keras.applications.resnet.preprocess_input(image)
    except:
        print(f"Invalid image format, skipping: {image_path}")
        return None
    return image

def create_dataset(df):
    image_dataset = tf.data.Dataset.from_tensor_slices(df['filename'])
    angle_dataset = tf.data.Dataset.from_tensor_slices(df['steering_angle'])
    image_dataset = image_dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    image_dataset = image_dataset.apply(tf.data.experimental.ignore_errors())
    dataset = tf.data.Dataset.zip((image_dataset, angle_dataset))

    return dataset

validation_split = 0.1
df = df.sample(frac=1).reset_index(drop=True)
val_df = df[:int(validation_split*len(df))]
train_df = df[int(validation_split*len(df)):]

train_dataset = create_dataset(train_df)
val_dataset = create_dataset(val_df)

batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

model = resnet_model((224, 224, 3))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='mean_absolute_error')

Instructions for updating:
Use `tf.data.Dataset.ignore_errors` instead.


94765736/94765736 [==============================] - 3s 0us/step


In [8]:

# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# Define learning rate scheduler
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)

# Fit the model
model.fit(train_dataset, validation_data=val_dataset, epochs=50, callbacks=[early_stopping, lr_scheduler])

Epoch 1/50
1278/1278 [==============================] - 223s 160ms/step - loss: 14.8169 - val_loss: 14.8420 - lr: 1.0000e-05
Epoch 2/50
1278/1278 [==============================] - 201s 154ms/step - loss: 14.0919 - val_loss: 13.9874 - lr: 1.0000e-05
Epoch 3/50
1278/1278 [==============================] - 204s 156ms/step - loss: 13.3133 - val_loss: 13.0045 - lr: 1.0000e-05
Epoch 4/50
1278/1278 [==============================] - 205s 158ms/step - loss: 12.5514 - val_loss: 12.2076 - lr: 1.0000e-05
Epoch 5/50
1278/1278 [==============================] - 206s 157ms/step - loss: 11.8740 - val_loss: 11.4191 - lr: 1.0000e-05
Epoch 6/50
1278/1278 [==============================] - 197s 152ms/step - loss: 11.2337 - val_loss: 10.8019 - lr: 1.0000e-05
Epoch 7/50
1278/1278 [==============================] - 199s 153ms/step - loss: 10.6967 - val_loss: 10.3315 - lr: 1.0000e-05
Epoch 8/50
1278/1278 [==============================] - 205s 156ms/step - loss: 10.2444 - val_loss: 9.8461 - lr: 1.0000e-05
E

## Testing for a random image

In [13]:
import random
from tensorflow.keras.preprocessing import image as keras_image
import numpy as np

def load_image(image_path, target_size=(224, 224)):
    img = keras_image.load_img(image_path, target_size=target_size)
    img_tensor = keras_image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor = tf.keras.applications.resnet.preprocess_input(img_tensor)
    return img_tensor

# Randomly select an image from validation set
random_image_path = random.choice(val_df['filename'].tolist())

# Load the image
test_image = load_image(random_image_path)

# Use the model to predict the steering angle for the test image
predicted_angle = model.predict(test_image)

# Print out the predicted steering angle
print("Predicted steering angle: ", predicted_angle[0][0])

# If you want to compare this prediction to the actual angle, you could find that as follows:
actual_angle = val_df[val_df['filename'] == random_image_path]['steering_angle'].values[0]
print("Actual steering angle: ", actual_angle)


1/1 [==============================] - 0s 24ms/step
Predicted steering angle:  -5.4446106
Actual steering angle:  -2.62
